# 6장 데이터 로딩과 저장, 파일 형식

In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [3]:
!cat examples/ex1.csv

df = pd.read_csv("examples/ex1.csv")
df

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [4]:
pd.read_csv("examples/ex2.csv")

names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [5]:
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

    4) 여러 개의 공백으로 필드를 구분한 파일 
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [6]:
!cat "examples/ex3.txt
result = pd.read_csv("examples/ex3.txt", sep="\s+") # "\s+"-> spacebar가 복수개
result

zsh:1: unmatched "


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/var/folders/t4/c2fsqnmn48577z_ks0_82btm0000gn/T/ipykernel_69776/528989585.py:2: SyntaxWarning: invalid escape sequence '\s'
  result = pd.read_csv("examples/ex3.txt", sep="\s+") # "\s+"-> spacebar가 복수개


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트

In [7]:
!cat examples/ex4.csv
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


nulldata 처리 중요
    6) 누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력 
        * na_values 인수 - NA 값으로 처리할 값들의 목록

In [10]:
!cat examples/ex5.csv

result = pd.read_csv("examples/ex5.csv")
result
# 1) 누락된 값을 확인 : isna()함수 이용
pd.isna(result)

# 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("examples/ex5.csv", na_values=sentinels)
print(result)
result.isna()

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,True,False,False,True,False,False
2,False,False,False,False,False,True


    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 익을 파일의 크기

In [11]:
result = pd.read_csv("examples/ex6.csv", nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


### [예제]  ex6.csv 파일을 순회하면서 "key"열의 값을 세기

In [12]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 1.2. csv 텍스트 파일 저장하기
* 판다스에서 CSV 파일을 저장하기: df.to_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


In [13]:
data = pd.read_csv("examples/ex5.csv")
data #dataframe에 데이터 저장
data1 = data[['a','b','d']] # 인덱싱: 데이프레임의 이후 데이터만 추출해서 데이터프레임생성
data1
data1.to_csv("examples/out.csv") #데이프레임 to_csv(저장할 파일명)

# 2. HTML 데이터
## 2.1 웹 스크래핑 - read_html()

* read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


#### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서 읽어오기

In [14]:
import pandas as pd

tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
# print(type(tables))
len(tables)

failures = tables[0] # 인덱싱
# print(type(failures))
failures.head()

,Bank NameBank,CityCity,StateSt,CertCert,Acquiring InstitutionAI,Closing DateClosing,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543
3,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.","March 12, 2023",10540
4,Silicon Valley Bank,Santa Clara,CA,24735,First–Citizens Bank & Trust Company,"March 10, 2023",10539


In [16]:
# 1) 컬럼명 수정하기
# failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert',
       'AcquiringInstitutionAI', 'ClosingDate', 'FundFund']
failures.head(3)

,BankNameBank,CityCity,StateSt,CertCert,AcquiringInstitutionAI,ClosingDate,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543


In [17]:
# 2) 데이터 변환 : 문자열 -> datetime 객체 변환
close_timestamps = pd.to_datetime(failures["ClosingDate"])
close_timestamps.head()
#3) 년도별 데이터 개수 추출
close_timestamps.dt.year.value_counts()

ClosingDate
2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2017      8
2015      8
2023      5
2016      5
2020      4
2019      4
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, dtype: int64

#### [실습] : 네이버 주식 데이터 가져오기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [18]:
!pip install html5lib  lxml

  Using cached html5lib-1.1-py2.py3-none-any.whl.metadata (16 kB)
Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)


In [19]:
import pandas as pd

# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver",encoding='cp949')
type(df_list)
len(df_list)
# 데이터프레임 확인
# for df in df_list:
#     print(df)
df_list[1]

/Users/idaye/Desktop/bigdata/python_basic/venv/lib/python3.12/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,삼성전자,76200.0,100.0,+0.13%,100.0,4548974.0,5969783.0,55.87,9476830.0,35.76,4.15,NaN
2,2.0,SK하이닉스,172800.0,1200.0,+0.70%,5000.0,1257988.0,728002.0,54.41,2395336.0,-13.81,-15.61,NaN
3,3.0,LG에너지솔루션,372500.0,6000.0,-1.59%,500.0,871650.0,234000.0,4.85,63947.0,70.46,6.36,NaN
4,4.0,삼성바이오로직스,791000.0,11000.0,-1.37%,2500.0,562986.0,71174.0,12.00,14636.0,65.64,9.12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,49.0,아모레퍼시픽,140200.0,1500.0,-1.06%,500.0,82007.0,58493.0,27.78,92241.0,53.76,3.72,NaN
77,50.0,KODEX CD금리액티브(합성),1033640.0,100.0,+0.01%,0.0,80847.0,7822.0,0.01,307673.0,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# 데이터프레임 탐색
df = df_list[1]
#결측지 처리- 종목명
#매우중요 조건색인을 이용해서 불리언 인덱싱으로 새로운 데이터프레임 생성
#1. 조건색인
cond = df['종목명'].notnull() #notna()
#2. 조건색인을 데이터프레임에 넣어주기 -True 값만 추출
# data = df[df['종목명'].notnull()] # 조건색인
data =df[cond]
data
data.head()
#최종 결과를 데이프레임 형태로 저장 - df.to_csv(파일명)
data.to_csv("Naver_kospi.csv")

## 3.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용
  * requests.get()
  * requests.get().status_code
  * requests.get().text


### [예제] 

In [22]:
# 1. requests 모듈 설치하기
# !pip install requests
# # 2. requests 모듈 임포트하기
import requests
#3. url 가져오기
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
print(resp.status_code)
resp.text
#원하는 데이터 처리 방법
# 5. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
type(data)
len(data)
#6. json 내용 확인
data[0]
#7.json 딕셔너리
# data[0]['title']
data[0].keys() # 키 정보 출력
# 8. 딕셔너리를 DataFrame으로 생성하기
issues = pd.DataFrame(data, columns=["number", "title",
                                      "labels", "state"])
issues.head()
#데이터프레임을 저장
issues.to_csv("pandas_issues.csv")


200


### [문제] 네이버 블로그의 키워드 관련 검색 결과 텍스트 데이터 가져오기 - requests 모듈 사용시 한계점

In [ ]:
import requests
# url = https://section.blog.naver.com/Search/Post.naver?\
# pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC
# response = requests.get(url)

params = {
    'pageNo' : 1,
    'rangeType' : 'ALL',
    'orderBy' : 'sim',
    'keyword' : '파이썬'
}



# response = requests.get('https://section.blog.naver.com/Search/Post.naver?', params=params)

print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

## 3.3 웹 스크래핑 - beautifulsoup
* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 
  * soup.select_one()
  * soup.select()
  * get_text()
* 개발자 도구 사용

### [문제] 네이버 지식iN의 검색어 관련 검색 결과 제목 가져오기

In [23]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    #웹 페이지 원하는 데이터 정보로 추출
    # 1. html -> beautifulsoup 객체로 변환
    soup = BeautifulSoup(html, 'html.parser')

    # 제목 한개 선택하여 텍스트 가져오기
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    print(title.get_text())

    #첫번째 웹페이지의 모든 제목들을 선택하여 텍스트 리스트 가져오기
    # 제목 여러 개 선택하여 텍스트 리스트 가져오기
    titles = soup.select('#s_content > div.section > ul > li:nth-child(n) > dl > dt > a')
    print(type(title))
    for title in titles:
        print(title.get_text())
else : 
    print(response.status_code)



파이썬이 미래에 도움이 될까요?
<class 'bs4.element.Tag'>
파이썬이 미래에 도움이 될까요?
중1 파이썬 독학
파이썬 사용
파이썬 ;
파이썬 질문
파이썬 질문이요
파이썬 해커 되는법 파이썬 해킹에 관해서
파이썬과 자바
파이썬 무역학과, 무역회사
파이썬강좌 독학할때 뭐보나요??


## 3.4 웹 스크래핑 - Selenium을 이용하여 사이트 자동화하기
* selenium 사용법
  1. 시스템의 OS 및 Chrome 브라우저의 버전 확인
  2. Chrome 드라이버 다운로드 및 설치
  3. 크롬 드라이버을 프로젝트 폴더에 저장 - 크롬 드라이버의 설치 경로
     * from selenium import webdriver
     * driver = webdriver.Chrome()
* 동적으로 웹 페이지 정보 추출하기
     * driver.get("웹주소") 
  

### [예제] Chrome driver 사용 코드

In [8]:
!pip install selenium

  Using cached selenium-4.19.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached trio-0.25.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached selenium-4.19.0-py3-none-any.whl (10.5 MB)
Using cached trio-0.25.0-py3-none-any.whl (467 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)


In [9]:
from selenium import webdriver
# Chrome 브라우저 실행
driver = webdriver.Chrome()
# 웹페이지 열기
url = "https://www.google.com"
driver.get(url)

### [문제] 구글 검색창에서 "openai'로 검색한 결과를 제목과 url로 출력하는 자동 프로그램 작성


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# 1. Chrome 브라우저 실행
driver = webdriver.Chrome()

# 2. Google 홈페이지 열기
url = "https://www.google.com"
driver.get(url)

# 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(5)

try:
    # 검색 상자 요소 찾기
    search_box = driver.find_element(By.NAME, "q")

    # 검색어 입력
    search_query = "OpenAI"
    search_box.send_keys(search_query)

    # Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN)

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(5)

    # 검색 결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g")

    # 검색 결과의 제목과 URL 인쇄
    for result in search_results:
        title = result.find_element(By.CSS_SELECTOR, "h3").text
        url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        print("제목:", title)
        print("URL:", url)
        print()

except Exception as e:
    print("오류가 발생했습니다:", e)

finally:
    # 브라우저 창 닫기
    driver.quit()


제목: OpenAI
오류가 발생했습니다: 'str' object has no attribute 'get_attribute'


In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()
driver.get("https://example.com")
elements_with_a = driver.find_elements(By.CSS_SELECTOR, "a") 
print("특정 a 요소:",elements_with_a)
print("특정 a 요소의 개수:", len(elements_with_a))
print("특정 a 요소의 속성 href 값 가져오기:",elements_with_a[0].get_attribute("href")) 
driver.quit()

특정 a 요소: [<selenium.webdriver.remote.webelement.WebElement (session="52388db460389733c3806c00f57f0890", element="f.2CD62629F5815B6FCD3944E0964762A2.d.3BA2A6317AF62D18EF033D2186544616.e.2")>]
특정 a 요소의 개수: 1
특정 a 요소의 속성 href 값 가져오기: https://www.iana.org/domains/example


## [응용문제] 시가총액이 높은 상위 500개 기업의 데이터를 가져오기
1~50 페이지 중 10개 페이지만 데이터 읽어오기
10개 페이지를 하나의 데이터로 합치기
위결과를 csv파일로 저장


from selenium import webdriver
import pandas as pd

In [26]:
#Chrome 브라우저를 열고 웹 페이지로 이동
import time
driver = webdriver.Chrome()

#시가총액 상위 500개 기업 데이터를 가져올 URL
url = 'https://finance.naver.com/sise/sise_market_sum.naver?&page='

#10개 페이지를 저장할 리스트
kospi500 = []

# 10개 페이지에 대해 반복하여 데이터 수집
for num in range(1,11): #페이지 번호 제어
    full_url = url +str(num)
    driver.get(full_url) # 페이지 이동
    time.sleep(2) #페이지가 로드될 때까지 기다림
    html = driver.page_source #페이지 html 가져오기
    data = pd.read_html(html) #html을 데이터프레임으로 읽기 시도
    df = data[1] # 두번째 테이블에 중복 데이터가 있음
    df = df[df['종목명'].notnull()]
    kospi500.append(df)

    combimed_df = pd.concat(kospi500)

    combimed_df.to_csv("combimed_df.csv")

    driver.quit()

/var/folders/t4/c2fsqnmn48577z_ks0_82btm0000gn/T/ipykernel_69776/31977513.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(html) #html을 데이터프레임으로 읽기 시도


MaxRetryError: HTTPConnectionPool(host='localhost', port=56307): Max retries exceeded with url: /session/7ac58e88958d3f06b1280cfc4c8cdaa7/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x125ce3aa0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
#Chrome 웹 라이버 실행
driver = webdriver.Chrome()

In [ ]:
# 페이지 로딩 시간
time.sleep(2)

In [ ]:
# target 웹페잊 url - 사전 작업
#삼성전자 일별 시세 정보 페이지
samsung_url = http://finance.naver.com/item/sise_daynaver?code=005930&page=1